# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [11]:
# TODO: import needed libraries
import numpy as np
import pandas as pd
import gensim

Load the data in the file `random_headlines.csv`

In [12]:
# TODO: load the dataset
df = pd.read_csv('random_headlines.csv')
print(df.shape)
df.head()

(20000, 2)


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [13]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [15]:
# TODO: Preprocess the input data

from nltk.tokenize import word_tokenize

df['tokenize'] = df['headline_text'].apply(lambda row: word_tokenize(row))
df['tokenize']
#punctuation removal

df['alphanumeric'] = df['tokenize'].apply(lambda row: [ 
    word for word in row if word.isalpha()
])

# stopword removal
from nltk.corpus import stopwords

stop = stopwords.words('english')
df['stop'] = df['alphanumeric'].apply(lambda row: [
   word for word in row if word not in stop
])

# stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
df['stemmed'] = df['stop'].apply(lambda row: [
    stemmer.stem(word) for word in row
])

df['stemmed'].head()

0    [ute, driver, hurt, intersect, crash]
1                       [die, cycl, accid]
2          [bumper, oliv, harvest, expect]
3    [replica, replac, northernmost, sign]
4          [wood, target, perfect, season]
Name: stemmed, dtype: object

Now use Gensim to compute a BOW

In [18]:
# TODO: Compute the BOW using Gensim
from gensim.corpora import Dictionary

dictionary = Dictionary(df['stemmed'])

BOW = [dictionary.doc2bow(document) for document in df['stemmed']]

print(np.shape(BOW))
BOW[:2]


(20000,)


C:\Users\Ollie\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1)]]

Compute the TF-IDF using Gensim

In [20]:
# TODO: Compute TF-IDF
from gensim.models import TfidfModel

tfidf_model = TfidfModel(BOW)
tfidf = tfidf_model[BOW]

print(np.shape(tfidf))
tfidf

(20000,)


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [24]:
# TODO: Compute LSA
from gensim.models import LsiModel

lsi_model = LsiModel(BOW, id2word=dictionary, num_topics=4)

For each of the topic, show the most significant words.

In [25]:
# TODO: Print the 3 or 4 most significant words of each topic
lsi_model.print_topics(num_topics=4, num_words=3)

[(0, '0.752*"polic" + 0.405*"man" + 0.208*"charg"'),
 (1, '-0.669*"man" + 0.575*"polic" + -0.328*"charg"'),
 (2, '-0.654*"new" + -0.296*"plan" + -0.242*"say"'),
 (3, '-0.702*"new" + 0.345*"say" + 0.335*"plan"')]

The model outputs as predicte the common topicssuch as police charging are the same 

What do you think about those results?

Now let's try to use LDA instead of LSA using Gensim

In [33]:
# TODO: Compute LDA
from gensim.models import LdaModel

lda_model = LdaModel(BOW, id2word=dictionary, num_topics=4)

In [46]:
# TODO: print the most frequent words of each topic

lda_model.print_topics(num_topics=4, num_words=6)

[(0,
  '0.008*"say" + 0.007*"polic" + 0.007*"health" + 0.006*"council" + 0.006*"plan" + 0.005*"govt"'),
 (1,
  '0.011*"interview" + 0.008*"charg" + 0.006*"market" + 0.006*"new" + 0.005*"man" + 0.005*"plan"'),
 (2,
  '0.009*"win" + 0.006*"polic" + 0.005*"farmer" + 0.005*"australia" + 0.004*"new" + 0.004*"world"'),
 (3,
  '0.010*"man" + 0.009*"kill" + 0.008*"polic" + 0.006*"fire" + 0.006*"home" + 0.006*"jail"')]

In [39]:
#install plyLDAvis
!pip install pyLDAvis

Now, how does it work with LDA?

Let's make some visualization of the LDA results using pyLDAvis.

In [47]:
# TODO: show visualization results of the LDA
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim_models.prepare(lda_model, BOW, dictionary)
vis

C:\Users\Ollie\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.095999 -0.020006       1        1  26.070850
3     -0.184145 -0.139050       2        1  25.952189
1      0.160762 -0.051996       3        1  24.212654
2     -0.072617  0.211052       4        1  23.764307, topic_info=          Term        Freq       Total Category  logprob  loglift
130  interview  283.000000  283.000000  Default  30.0000  30.0000
500        win  261.000000  261.000000  Default  29.0000  29.0000
688       kill  265.000000  265.000000  Default  28.0000  28.0000
138     health  177.000000  177.000000  Default  27.0000  27.0000
238       jail  160.000000  160.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
55         say   99.534034  383.365709   Topic4  -5.5010   0.0885
494        sex   64.407381  105.849349   Topic4  -5.9363   0.9402
305        set   75.492676  195.851333   Topic4  -5.7775   0.4837
124         us   78.116147  263.242818   Topic4  -5.7433   0.2221
640       back   74.675113  229.551424   Topic4  -5.7884   0.3140

[319 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
496       2  0.008963      abc
496       3  0.985938      abc
496       4  0.008963      abc
5         1  0.973356    accid
5         3  0.015450    accid
...     ...       ...      ...
151       1  0.008145    world
151       2  0.138471    world
151       4  0.855261    world
2641      3  0.990466    worri
3465      3  0.992050  zealand

[609 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.